# Built-in Training and Evaluation Loops

Let us start for the most basic workflow.

In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers

# create a model within a function to re-use it
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

# load data and reserve some part for validation
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss     ="sparse_categorical_crossentropy",
    metrics  =["accuracy"]
)
model.fit(
    train_images, train_labels,
    epochs=3,
    validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8623 - loss: 0.4506 - val_accuracy: 0.9588 - val_loss: 0.1407
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9526 - loss: 0.1638 - val_accuracy: 0.9682 - val_loss: 0.1144
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9643 - loss: 0.1257 - val_accuracy: 0.9725 - val_loss: 0.1009
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9692 - loss: 0.1024
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


There are a couple of ways you can customize this simple workflow:
1. Provide your own custom metrics.
2. Pass callbacks to the `fit()` method to schedule actions to be taken at specific points during training.

Commonly used metrics for classification and regression are already part of the built-in `keras.metrics` module, and most of the time that’s what you will use. But if you’re doing anything out of the ordinary, you will need to be able
to write your own metrics.

A Keras metric is a subclass of the `keras.metrics.Metric` class. Like layers, a metric has an internal state stored in TensorFlow variables. Unlike layers, these variables aren’t updated via backpropagation, so you have to write the state-update logic yourself, which happens in the `update_state()` method.

In [3]:
# Implementing a custom metric by subclassing the Metric class

import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    # Define the state variables in the constructor. Like for layers, you
    # have access to the add_weight() method.
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
        name="total_samples", initializer="zeros", dtype="int32")

    # Implement the state update logic in update_state(). The y_true argument
    # is the targets (or labels) for one batch, while y_pred represents the
    # corresponding predictions from the model.
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    # use the result() method to return the current value of the metric:
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    # expose a way to reset the metric state without having to reinstantiate it
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [4]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss     ="sparse_categorical_crossentropy",
    metrics  =["accuracy", RootMeanSquaredError()]
)
model.fit(
    train_images, train_labels,
    epochs=3,
    validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8635 - loss: 0.4483 - rmse: 0.4375 - val_accuracy: 0.9593 - val_loss: 0.1426 - val_rmse: 0.2490
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9534 - loss: 0.1586 - rmse: 0.2661 - val_accuracy: 0.9681 - val_loss: 0.1116 - val_rmse: 0.2186
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9635 - loss: 0.1316 - rmse: 0.2389 - val_accuracy: 0.9719 - val_loss: 0.1001 - val_rmse: 0.2075
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9694 - loss: 0.1103 - rmse: 0.2152


Now, let us see an example of how to use callbacks during model training.

In [5]:
# Callbacks are passed to the model via the callbacks argument in fit(), which takes a list of
# callbacks. One can pass any number of callbacks.
callbacks_list = [
    keras.callbacks.EarlyStopping( # Interrupts training when improvement stops
        monitor="val_accuracy", # Monitors the model’s validation accuracy
        patience=2, # Interrupts training when accuracy has stopped improving for two epochs
    ),
    keras.callbacks.ModelCheckpoint( # Saves the current weights after every epoch
        filepath="checkpoint_path.keras", # Path to the destination model file
        monitor="val_loss",  # These two arguments mean you won’t overwrite the model file unless val_loss
        save_best_only=True, # has improved, which allows you to keep the best model seen during training.
    )
]
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss     ="sparse_categorical_crossentropy",
    metrics  =["accuracy"] # You monitor accuracy, so it should be part of the model’s metrics.
)
model.fit(
    train_images, train_labels,
    epochs=10,
    callbacks=callbacks_list,
    validation_data=(val_images, val_labels)
)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8638 - loss: 0.4468 - val_accuracy: 0.9594 - val_loss: 0.1413
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9534 - loss: 0.1627 - val_accuracy: 0.9682 - val_loss: 0.1144
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9624 - loss: 0.1324 - val_accuracy: 0.9730 - val_loss: 0.1020
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9692 - loss: 0.1084 - val_accuracy: 0.9751 - val_loss: 0.1004
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9714 - loss: 0.1066 - val_accuracy: 0.9776 - val_loss: 0.0920
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9744 - loss: 0.0954 - val_accuracy: 0.9784 - val_loss: 0.0972
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9758 - loss: 0.0865 - val_accuracy: 0.9787 - val_loss: 0.0931
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9776 - loss: 0.0814 - 

A good way to have more control on the experiments is to "see" what is happening during the training process. A good tool to accomplish this is TensorBoard. Let us see how to use it.

In [7]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
tensorboard = keras.callbacks.TensorBoard(
    log_dir="/mnt/0A2AAC152AABFBB7/sideProjects/deepLearning/images",
)
model.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(val_images, val_labels),
    callbacks=[tensorboard]
)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8653 - loss: 0.4543 - val_accuracy: 0.9570 - val_loss: 0.1500
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9515 - loss: 0.1675 - val_accuracy: 0.9681 - val_loss: 0.1132
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9615 - loss: 0.1349 - val_accuracy: 0.9715 - val_loss: 0.1096
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9666 - loss: 0.1145 - val_accuracy: 0.9727 - val_loss: 0.1003
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9722 - loss: 0.1019 - val_accuracy: 0.9766 - val_loss: 0.0923
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9742 - loss: 0.0904 - val_accuracy: 0.9771 - val_loss: 0.1000
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9759 - loss: 0.0844 - val_accuracy: 0.9757 - val_loss: 0.0989
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9775 - loss: 0.0824 - 

In [8]:
%load_ext tensorboard
%tensorboard --logdir /mnt/0A2AAC152AABFBB7/sideProjects/deepLearning/images